In [1]:
pip install -U torch transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 34.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EvalPrediction
from datasets import load_dataset, DatasetDict
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_ckpt = "Alibaba-NLP/gte-base-en-v1.5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)

cuda


In [5]:
dataset = DatasetDict({
    "train": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/train_under.csv", split="train"),
    "validation": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/val_under.csv", split="train"),
    "test": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/test.csv", split="train"),
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 8923
    })
    validation: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 2231
    })
    test: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 3462
    })
})


In [7]:
dataset = dataset.map(
    lambda x: {
        "text" : x["corrected_text"]
    },
    remove_columns = ["corrected_text", "length", "ratio_err"]
)

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 8923
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 2231
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 3462
    })
})


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [10]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [11]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [12]:
tokenizer.tokenize("Hello world, how are u ?", add_special_tokens=True)

['[CLS]', 'hello', 'world', ',', 'how', 'are', 'u', '?', '[SEP]']

In [13]:
tokenizer.pad_token

'[PAD]'

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
dataset = dataset.map(lambda x: {"labels": x["labels"] - 1})

Map:   0%|          | 0/8923 [00:00<?, ? examples/s]

Map:   0%|          | 0/2231 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           trust_remote_code=True,
                                                           num_labels=6).to(device)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/57.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

Some weights of NewForSequenceClassification were not initialized from the model checkpoint at Alibaba-NLP/gte-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
print(model)

NewForSequenceClassification(
  (new): NewModel(
    (embeddings): NewEmbeddings(
      (word_embeddings): Embedding(30528, 768, padding_idx=0)
      (rotary_emb): NTKScalingRotaryEmbedding()
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): NewEncoder(
      (layer): ModuleList(
        (0-11): 12 x NewLayer(
          (attention): NewAttention(
            (qkv_proj): Linear(in_features=768, out_features=2304, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (o_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (mlp): NewGatedMLP(
            (up_gate_proj): Linear(in_features=768, out_features=6144, bias=False)
            (down_proj): Linear(in_features=3072, out_features=768, bias=True)
            (act_fn): GELUActivation()
            (hidden_dropout): Dropout(p=0.1, inplace=False)
          )
          (attn_ln): LayerNorm((768,), 

In [18]:
def compute_metrics(preds: EvalPrediction):
    """
    Compute metrics for the task

    Args:
        preds {EvalPrediction}: the predictions from the model
    Returns:
        dict: a dictionary of metrics
    """

    labels = preds.label_ids
    preds = preds.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    weighted_precision = precision_score(labels, preds, average="weighted")
    weighted_recall = recall_score(labels, preds, average="weighted")
    weighted_f1 = f1_score(labels, preds, average="weighted")
    mcc = matthews_corrcoef(labels, preds)

    macro_precision = precision_score(labels, preds, average="macro")
    macro_recall = recall_score(labels, preds, average="macro")
    macro_f1 = f1_score(labels, preds, average="macro")

    micro_precision = precision_score(labels, preds, average="micro")
    micro_recall = recall_score(labels, preds, average="micro")
    micro_f1 = f1_score(labels, preds, average="micro")

    return {
        "accuracy": acc,
        "weighted_precision": weighted_precision,
        "weighted_recall": weighted_recall,
        "weighted_f1": weighted_f1,
        "mcc": mcc,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1
    }

In [19]:
BATCH_SIZE = 2
LR = 2e-5

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models_lazy_teacherv2",
    num_train_epochs=6,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    learning_rate=LR,
    evaluation_strategy="steps",
    bf16=True,
    eval_steps=250,
    report_to="tensorboard",
    lr_scheduler_type="cosine"
)

In [20]:
weights = dataset["train"].to_pandas()["labels"].value_counts(normalize=False).sort_index().values
weights = 1 / weights

In [21]:
weights = torch.tensor(weights)

In [22]:
weights

tensor([0.0012, 0.0004, 0.0004, 0.0004, 0.0016, 0.0100], dtype=torch.float64)

In [26]:
class WeightedTrainer(Trainer):

    classes_weights = torch.tensor([0.0012, 0.0004, 0.0004, 0.0004, 0.0016, 0.0100]).to(device)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.classes_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [27]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Weighted Precision,Weighted Recall,Weighted F1,Mcc,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1
250,1.436500,1.219871,0.475123,0.516052,0.475123,0.466097,0.327053,0.425474,0.418764,0.368215,0.475123,0.475123,0.475123
500,1.007900,0.987717,0.609144,0.618389,0.609144,0.604078,0.482575,0.665968,0.496969,0.503253,0.609144,0.609144,0.609144
750,0.848200,0.949426,0.590766,0.619855,0.590766,0.586296,0.470348,0.609623,0.538324,0.534185,0.590766,0.590766,0.590766
1000,0.898300,1.007570,0.624832,0.619608,0.624832,0.615457,0.500003,0.521458,0.483443,0.494254,0.624832,0.624832,0.624832
1250,0.719300,0.916635,0.617212,0.623370,0.617212,0.614901,0.500754,0.581797,0.588338,0.577304,0.617212,0.617212,0.617212
1500,0.703100,0.960820,0.631555,0.630174,0.631555,0.625731,0.512511,0.608762,0.549458,0.567178,0.631555,0.631555,0.631555
1750,0.613200,1.076522,0.619901,0.624701,0.619901,0.618061,0.500015,0.588765,0.576881,0.574738,0.619901,0.619901,0.619901
2000,0.451900,1.157668,0.597938,0.609785,0.597938,0.599094,0.476435,0.564208,0.588833,0.567127,0.597938,0.597938,0.597938
2250,0.464600,1.196907,0.625280,0.633851,0.625280,0.624751,0.508796,0.625024,0.573126,0.576606,0.625280,0.625280,0.625280
2500,0.273800,1.302309,0.635141,0.639521,0.635141,0.635559,0.518941,0.631814,0.601553,0.610647,0.635141,0.635141,0.635141


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on th

TrainOutput(global_step=3342, training_loss=0.6254812902065896, metrics={'train_runtime': 2020.2308, 'train_samples_per_second': 26.501, 'train_steps_per_second': 1.654, 'total_flos': 1.4167885925674548e+16, 'train_loss': 0.6254812902065896, 'epoch': 5.991931869116988})

In [29]:
trainer.save_model("/content/drive/MyDrive/models_lazy_teacherv2")

In [31]:
tokenizer.save_pretrained("/content/drive/MyDrive/models_lazy_teacherv2")

('/content/drive/MyDrive/models_lazy_teacherv2/tokenizer_config.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/special_tokens_map.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/vocab.txt',
 '/content/drive/MyDrive/models_lazy_teacherv2/added_tokens.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/tokenizer.json')

In [32]:
trainer.evaluate(eval_dataset=dataset["test"])

{'eval_loss': 1.4655600786209106,
 'eval_accuracy': 0.6158290005777007,
 'eval_weighted_precision': 0.6169248568179411,
 'eval_weighted_recall': 0.6158290005777007,
 'eval_weighted_f1': 0.6138433105833321,
 'eval_mcc': 0.47879943362751193,
 'eval_macro_precision': 0.5979385218479537,
 'eval_macro_recall': 0.5414934316628028,
 'eval_macro_f1': 0.5580960470307481,
 'eval_micro_precision': 0.6158290005777007,
 'eval_micro_recall': 0.6158290005777007,
 'eval_micro_f1': 0.6158290005777007,
 'eval_runtime': 17.9621,
 'eval_samples_per_second': 192.739,
 'eval_steps_per_second': 48.213,
 'epoch': 5.991931869116988}

In [41]:
trainer.model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-2500").to(device)

The repository for /content/drive/MyDrive/models_lazy_teacherv2/checkpoint-2500 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/models_lazy_teacherv2/checkpoint-2500.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The repository for /content/drive/MyDrive/models_lazy_teacherv2/checkpoint-2500 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/models_lazy_teacherv2/checkpoint-2500.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [42]:
trainer.evaluate(eval_dataset=dataset["test"])

{'eval_loss': 1.3012906312942505,
 'eval_accuracy': 0.6164067013287118,
 'eval_weighted_precision': 0.6193687887673887,
 'eval_weighted_recall': 0.6164067013287118,
 'eval_weighted_f1': 0.6153898525056224,
 'eval_mcc': 0.48104025263293654,
 'eval_macro_precision': 0.5859024827261887,
 'eval_macro_recall': 0.5547844801655557,
 'eval_macro_f1': 0.5627802821894966,
 'eval_micro_precision': 0.6164067013287118,
 'eval_micro_recall': 0.6164067013287118,
 'eval_micro_f1': 0.6164067013287118,
 'eval_runtime': 35.6616,
 'eval_samples_per_second': 97.079,
 'eval_steps_per_second': 24.284,
 'epoch': 5.991931869116988}